In [ ]:
# Generate mock data for 100 students with real names and 4 ranked choices for each session
students_data = {
    'student_id': range(1, 101),
    'name': np.random.choice(
        ['Alice', 'Bob', 'Charlie', 'David', 'Eve', 'Frank', 'Grace', 'Hannah', 'Ivy', 'Jack',
         'Kara', 'Liam', 'Mia', 'Nina', 'Oscar', 'Paul', 'Quinn', 'Rachel', 'Sam', 'Tina',
         'Uma', 'Victor', 'Wendy', 'Xander', 'Yara', 'Zach', 'Bella', 'Chris', 'Diana', 'Ethan',
         'Fiona', 'George', 'Holly', 'Ian', 'Jill', 'Kyle', 'Laura', 'Matt', 'Nate', 'Olivia',
         'Pete', 'Queen', 'Ryan', 'Sara', 'Tom', 'Ursula', 'Vince', 'Will', 'Xena', 'Yvonne',
         'Zane', 'Alyssa', 'Brian', 'Cathy', 'Derek', 'Emma', 'Felix', 'Gloria', 'Harry', 'Iris',
         'Jake', 'Kelly', 'Leo', 'Mona', 'Nick', 'Owen', 'Piper', 'Quentin', 'Rose', 'Steve', 
         'Tara', 'Ulysses', 'Violet', 'Wade', 'Ximena', 'Yusuf', 'Zelda', 'Amber', 'Brad', 'Cindy', 
         'Dean', 'Ella', 'Franklin', 'Gina', 'Hector', 'Isla', 'Jon', 'Kate', 'Louis', 'Maggie', 
         'Nolan', 'Opal', 'Paula', 'Quincy', 'Rob', 'Sophie', 'Trent', 'Uriah', 'Vera', 'Winston'],
        size=100, replace=False),
    'grade': np.random.randint(1, 9, size=100)
}

# Define the periods
periods = ['Monday AM', 'Monday PM', 'Tuesday AM', 'Tuesday PM', 'Wednesday AM', 'Wednesday PM', 'Thursday AM', 'Thursday PM', 'Friday AM', 'Friday PM']

# Add 4 ranked choices for each period
for period in periods:
    for choice in range(1, 5):
        students_data[f'choice{choice}_{period.replace(" ", "_").lower()}'] = np.random.randint(1, 21, size=100)

students_df = pd.DataFrame(students_data)

# Generate mock data for 20 activities with real names and sessions
activity_names = [
    'Art', 'Science', 'Math Games', 'Robotics', 'Music', 'Sports', 
    'Painting', 'Cooking', 'Reading', 'Drama', 'Chess', 'Football', 
    'Rock Climbing', 'Dance', 'Photography', 'Coding', 'Gardening', 'Writing', 
    'Astronomy', 'Yoga'
]

activities_data = {
    'activity_id': range(1, 21),
    'name': activity_names,
    'min_grade': np.random.randint(1, 5, size=20),
    'max_grade': np.random.randint(4, 9, size=20),
    'max_capacity': np.random.randint(5, 15, size=20),
    'sessions': np.random.randint(1, 5, size=20),
    'available_sessions': [",".join(np.random.choice(periods, size=np.random.randint(1, 6), replace=False)) for _ in range(20)]
}

activities_df = pd.DataFrame(activities_data)

# Generate mock data for 30 conflicts
conflicts_data = {
    'student_id1': np.random.randint(1, 101, size=30),
    'student_id2': np.random.randint(1, 101, size=30),
}

# Ensure no self-conflicts
conflicts_df = pd.DataFrame(conflicts_data)
conflicts_df = conflicts_df[conflicts_df['student_id1'] != conflicts_df['student_id2']]

# Generate mock data for group conflicts
group_conflicts_data = {
    'group_id': np.repeat(range(1, 11), 4),
    'student_id': np.random.randint(1, 101, size=40)
}
group_conflicts_df = pd.DataFrame(group_conflicts_data)

# Save data to CSV
students_df.to_csv('/mnt/data/students.csv', index=False)
activities_df.to_csv('/mnt/data/activities.csv', index=False)
conflicts_df.to_csv('/mnt/data/conflicts.csv', index=False)
group_conflicts_df.to_csv('/mnt/data/group_conflicts.csv', index=False)

students_df.head(), activities_df.head(), conflicts_df.head(), group_conflicts_df.head()


: 